In [2]:
from miditok import Structured    
from miditok.utils import get_midi_programs
from miditoolkit import MidiFile
from pathlib import Path

tokenizer = Structured()
midi = MidiFile('/home/mm/midi/snes_clean/ff3end.mid')

tokens = tokenizer(midi)  
converted_back_midi = tokenizer(tokens, get_midi_programs(midi))
tokens

[TokSequence(tokens=['Pitch_65', 'Velocity_99', 'Duration_2.0.8', 'TimeShift_0.0.1', 'Pitch_68', 'Velocity_99', 'Duration_0.4.8', 'TimeShift_0.4.8', 'Pitch_73', 'Velocity_99', 'Duration_0.4.8', 'TimeShift_0.4.8', 'Pitch_77', 'Velocity_99', 'Duration_0.4.8', 'TimeShift_0.4.8', 'Pitch_80', 'Velocity_99', 'Duration_0.4.8', 'TimeShift_0.4.8', 'Pitch_63', 'Velocity_99', 'Duration_2.0.8', 'TimeShift_0.0.1', 'Pitch_79', 'Velocity_99', 'Duration_0.4.8', 'TimeShift_0.4.8', 'Pitch_75', 'Velocity_99', 'Duration_0.4.8', 'TimeShift_0.4.8', 'Pitch_70', 'Velocity_99', 'Duration_0.4.8', 'TimeShift_0.4.8', 'Pitch_79', 'Velocity_99', 'Duration_0.4.8', 'TimeShift_0.4.8', 'Pitch_61', 'Velocity_99', 'Duration_4.0.4', 'TimeShift_0.0.1', 'Pitch_65', 'Velocity_75', 'Duration_6.0.4', 'TimeShift_0.0.1', 'Pitch_77', 'Velocity_99', 'Duration_1.0.8', 'TimeShift_1.0.8', 'Pitch_73', 'Velocity_99', 'Duration_3.0.8', 'TimeShift_3.0.8', 'Pitch_65', 'Velocity_99', 'Duration_2.0.8', 'TimeShift_0.0.1', 'Pitch_68', 'Veloci

In [8]:

# Converts MIDI files to tokens saved as JSON files
midi_paths = list(Path("..", "data", "maestro-v3.0.0").glob("**/*.midi"))
data_augmentation_offsets = [1, 1, 1]  # data augmentation on 2 pitch octaves, 1 velocity and 1 duration values
tokenizer.tokenize_midi_dataset(midi_paths, Path("/home/mm/midi/maestro_tokens/"),
                                data_augment_offsets=data_augmentation_offsets)

Performing data augmentation: 100%|██████████| 1276/1276 [00:55<00:00, 22.85it/s]


In [ ]:

# Constructs the vocabulary with BPE, from the tokenized files
tokenizer.learn_bpe(
    vocab_size=500,
    tokens_paths=list(Path("/home/mm/midi/maestro_tokens/").glob("**/*.json")),
    start_from_empty_voc=False,
)

# Saving our tokenizer, to retrieve it back later with the load_params method
tokenizer.save_params(Path("../data/tokenizer_params.json"))

In [ ]:
!ls /home/mm/midi/maestro_tokens/

In [2]:
import pandas as pd
import os
from pathlib import Path
from miditok import Structured
from miditok.utils import get_midi_programs
from tqdm import tqdm
from miditoolkit import MidiFile

os.makedirs(Path("../data/maestro_bpe/"), exist_ok=True)

df = pd.read_csv(Path("../data/maestro-v3.0.0.csv"))

tokenizer = Structured()
tokenizer.load_params(Path("../data/tokenizer_params.json"))

for split in ["train", "test", "validation"]:
    os.makedirs(Path(f"../data/maestro_bpe/{split}"), exist_ok=True)

for row in tqdm(df.itertuples(), total=len(df)):
    basename = os.path.basename(row.midi_filename)
    midi = MidiFile(Path("../data/maestro-v3.0.0", row.midi_filename))
    seq = tokenizer(midi)
    tokenizer.apply_bpe(seq)
    out_ = Path(f"../data/maestro_bpe/{row.split}/{basename}.json")
    tokenizer.save_tokens(seq, out_, get_midi_programs(midi))


100%|██████████| 1276/1276 [06:09<00:00,  3.45it/s]


In [8]:
import os
f = os.listdir(Path("/home/mm/midi/maestro_tokens_bpe/"))[0]
tokenizer.load_tokens(Path("/home/mm/midi/maestro_tokens_bpe/") / f)["ids"][0][:10]

[204, 27, 99, 256, 34, 332, 37, 426, 43, 427]